In [34]:
!pip3 install pyspark==3.0.0
import warnings
warnings.filterwarnings('ignore')


Defaulting to user installation because normal site-packages is not writeable


In [35]:
import os
import sys
from pyspark.sql import SparkSession
from typing import NamedTuple
from datetime import datetime
from functools import reduce

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.getOrCreate()

In [36]:
from pyspark import SparkContext, SparkConf
app_name = "Lab1"
spark.stop()
conf = SparkConf().setAppName(app_name).setMaster('local[1]')

sc = SparkContext(conf=conf)
sc

<SparkContext master=local[1] appName=Lab1>

In [37]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable


In [38]:
from pyspark import SparkContext, SparkConf
from typing import NamedTuple
from datetime import datetime
import numpy as np

In [39]:
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )
        
def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        yield Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
        )

In [40]:
trip_data = sc.textFile("trip.csv")
station_data = sc.textFile("station.csv")
trip_headers = trip_data.first()

In [41]:
station_headers = station_data.first()

In [42]:
trips = trip_data.filter(lambda row: row != trip_headers).map(lambda row: row.split(","))
stations = station_data.filter(lambda row: row != station_headers).map(lambda row: row.split(","))
print(trips.take(7), '\n')
print(stations.take(7))

[['4576', '63', '8/29/2013 14:13', 'South Van Ness at Market', '66', '8/29/2013 14:14', 'South Van Ness at Market', '66', '520', 'Subscriber', '94127'], ['4607', '70', '8/29/2013 14:42', 'San Jose City Hall', '10', '8/29/2013 14:43', 'San Jose City Hall', '10', '661', 'Subscriber', '95138'], ['4130', '71', '8/29/2013 10:16', 'Mountain View City Hall', '27', '8/29/2013 10:17', 'Mountain View City Hall', '27', '48', 'Subscriber', '97214'], ['4251', '77', '8/29/2013 11:29', 'San Jose City Hall', '10', '8/29/2013 11:30', 'San Jose City Hall', '10', '26', 'Subscriber', '95060'], ['4299', '83', '8/29/2013 12:02', 'South Van Ness at Market', '66', '8/29/2013 12:04', 'Market at 10th', '67', '319', 'Subscriber', '94103'], ['4927', '103', '8/29/2013 18:54', 'Golden Gate at Polk', '59', '8/29/2013 18:56', 'Golden Gate at Polk', '59', '527', 'Subscriber', '94109'], ['4500', '109', '8/29/2013 13:25', 'Santa Clara at Almaden', '4', '8/29/2013 13:27', 'Adobe on Almaden', '5', '679', 'Subscriber', '95

In [43]:
station_map = stations.mapPartitions(initStation)
trip_map= trips.mapPartitions(initTrip)

In [44]:
station_map.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))

In [45]:
trip_map.first()

Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 14, 13), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station_name='South Van Ness at Market', end_station_id='66', bike_id=520, subscription_type='Subscriber', zip_code='94127')

Найти велосипед с максимальным временем пробега:

In [46]:
bike_duration = trip_map.map(lambda x: (x.bike_id, x.duration)).reduceByKey(lambda x, y: x + y)
bike_duration_top = bike_duration.sortBy(lambda x: -x[1]).first()[0]

print(bike_duration_top)

535


Найти наибольшее геодезическое расстояние между станциями:


In [47]:
trips_stations = trip_map.filter(lambda trip: trip.start_station_id != trip.end_station_id)


trips_grouped = trips_stations.map(lambda trip: ((trip.start_station_id, trip.end_station_id), (trip.duration, 1)))\
    .reduceByKey(lambda acc, value: (acc[0] + value[0], acc[1] + value[1]))\
    .mapValues(lambda values: values[0] / values[1])


top_5_trips = trips_grouped.map(lambda x: (x[1], x[0]))\
    .sortByKey(False)\
    .take(5)


for trip in top_5_trips:
    print(trip[1], trip[0])

(26, '16') 229914.0
(32, '63') 179212.5
(80, '36') 169308.0
(66, '62') 156461.03603603604
(28, '2') 101207.5


Найти путь велосипеда с максимальным временем пробега через станции:

In [48]:
bike_path = trip_map.filter(lambda x: x.bike_id == bike_duration_top).sortBy(lambda x: x.start_date, ascending=True).map(lambda x: (x.start_station_name, x.end_station_name))
bike_path.first()

('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)')

Найти количество велосипедов в системе

In [49]:
count_of_bikes = trip_map.map(lambda x: x.bike_id).distinct().count()
print(count_of_bikes)

700


Найти пользователей потративших на поездки более 3 часов

In [50]:
users = trip_map.filter(lambda x: x.duration > (3 * 3600)).map(lambda x: x.zip_code).filter(lambda x: x != "").distinct()
users.take(5)


['58553', '94301', '94039', '94133', '93726']